<small>
Part of the InnovatED STEM and DroneBlocks Land, Air, and Sea Robotics Curriculum  
Licensed for educational use in schools only.  
Redistribution, commercial use, or resale is strictly prohibited.  
© 2025 InnovatED STEM & DroneBlocks. All rights reserved.
</small>

# Color Tracking Tutorial

This notebook will guide you through using the **color tracking** module to detect and track colored objects with a camera.

### Learning Objectives
- Understand how color detection works using HSV color space
- Learn to track specific colors in a video stream
- Process information about detected objects
- Create simple applications that respond to colored objects

## Setup

First, let's import the necessary modules and initialize our color tracker.

In [ ]:
import sys
import os
import time
import cv2
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output, display

# Add parent directory to the Python path
sys.path.insert(0, os.path.abspath('..'))

# Import our color tracking module
from color_tracking.color_tracker import ColorTracker, PID, find_largest_object

## Initialize the Color Tracker

Let's create a ColorTracker object and start it. This will initialize the camera and begin processing frames.

In [ ]:
# Create a ColorTracker object (camera index 0 is usually the default camera)
tracker = ColorTracker(camera_index=0)

# Start the tracker
tracker.start()

# Give it a moment to initialize
time.sleep(1)

## Display the Camera Feed

Let's see what the camera is seeing. We'll display both the raw camera feed and the processed image with detected objects highlighted.

In [ ]:
# Function to display images in the notebook
def display_frames(tracker, duration=5):
    """Display the raw and processed frames for a specified duration."""
    start_time = time.time()
    
    while time.time() - start_time < duration:
        # Get the frames
        raw_frame = tracker.get_frame()
        processed_frame = tracker.get_processed_frame()
        
        if raw_frame is None or processed_frame is None:
            print("No frames available yet.")
            time.sleep(0.5)
            continue
        
        # Convert from BGR to RGB for matplotlib
        raw_rgb = cv2.cvtColor(raw_frame, cv2.COLOR_BGR2RGB)
        processed_rgb = cv2.cvtColor(processed_frame, cv2.COLOR_BGR2RGB)
        
        # Create a figure with two subplots
        plt.figure(figsize=(12, 6))
        
        plt.subplot(1, 2, 1)
        plt.imshow(raw_rgb)
        plt.title('Raw Camera Feed')
        plt.axis('off')
        
        plt.subplot(1, 2, 2)
        plt.imshow(processed_rgb)
        plt.title(f'Processed Image - Tracking {tracker.current_color}')
        plt.axis('off')
        
        # Display the figure
        clear_output(wait=True)
        display(plt.gcf())
        plt.close()
        
        # Short delay
        time.sleep(0.1)

# Display frames for 5 seconds
display_frames(tracker, duration=5)

## Example 1: Changing the Tracked Color

By default, the tracker is set to track red objects. Let's change it to track a different color.

In [ ]:
# Change to track green objects
tracker.set_color('green')

# Display frames for 5 seconds
display_frames(tracker, duration=5)

### Challenge 1: Track a Different Color

Now it's your turn! Change the tracker to detect blue objects instead.
Fill in the missing code below:

In [ ]:
# Your code here:
# tracker.set_color(?)

# Display frames for 5 seconds
# display_frames(tracker, duration=5)

## Example 2: Getting Information About Detected Objects

Let's see how to get information about the objects that are detected.

In [ ]:
# Set color to red for this example
tracker.set_color('red')

# Wait a moment for the tracker to adjust
time.sleep(1)

# Get information about detected objects
objects = tracker.get_detected_objects()

# Print information about each detected object
print(f"Detected {len(objects)} red objects:")
for i, obj in enumerate(objects):
    print(f"Object {i+1}:")
    print(f"  Position: ({obj['x']:.1f}, {obj['y']:.1f})")
    print(f"  Radius: {obj['radius']:.1f} pixels")
    print(f"  Area: {obj['area']:.1f} square pixels")
    print()

### Challenge 2: Find the Largest Object

Write code to find and display information about only the largest detected object.
Use the `find_largest_object` function we imported earlier.

In [ ]:
# Your code here:
# objects = tracker.get_detected_objects()
# largest = find_largest_object(objects)

# if largest:
#     print("Largest object:")
#     print(f"  Position: ({largest['x']:.1f}, {largest['y']:.1f})")
#     print(f"  Radius: {largest['radius']:.1f} pixels")
#     print(f"  Area: {largest['area']:.1f} square pixels")
# else:
#     print("No objects detected.")

## Example 3: Creating a Custom Color Range

Sometimes you might want to track a specific color that's not in the predefined list. Let's see how to define a custom color range.

In [ ]:
# Define a custom color range for orange
# HSV values: [hue, saturation, value]
# Hue range: 0-180 in OpenCV (not 0-360)
orange_lower = [10, 100, 100]  # Lower bound of orange in HSV
orange_upper = [25, 255, 255]  # Upper bound of orange in HSV

# Add the custom color to the tracker
tracker.add_custom_color('orange', orange_lower, orange_upper)

# Set the tracker to track the new color
tracker.set_color('orange')

# Display frames for 5 seconds
display_frames(tracker, duration=5)

### Challenge 3: Create Your Own Custom Color

Now it's your turn to create a custom color! Try defining a range for purple or any other color you'd like to track.
Remember that in OpenCV, the HSV hue range is 0-180 (not 0-360 as in some other systems).

In [ ]:
# Your code here:
# Define HSV range for your custom color
# custom_lower = [?, ?, ?]
# custom_upper = [?, ?, ?]

# Add the custom color to the tracker
# tracker.add_custom_color('my_color', custom_lower, custom_upper)

# Set the tracker to track the new color
# tracker.set_color('my_color')

# Display frames for 5 seconds
# display_frames(tracker, duration=5)

## Example 4: Simple Object Following

Let's create a simple simulation of how a robot might follow a colored object. We'll use a PID controller to calculate how the robot should move to keep the object centered in the frame.

In [ ]:
# Set up PID controllers for x and y directions
pid_x = PID(p=0.1, i=0.01, d=0.05)
pid_y = PID(p=0.1, i=0.01, d=0.05)

# Set the color to track
tracker.set_color('red')

# Function to simulate robot following
def simulate_following(duration=10):
    start_time = time.time()
    
    # Get frame dimensions to set the center point
    frame = tracker.get_frame()
    if frame is None:
        print("No frame available yet.")
        return
        
    height, width = frame.shape[:2]
    center_x = width // 2
    center_y = height // 2
    
    # Set PID target to the center of the frame
    pid_x.SetPoint = center_x
    pid_y.SetPoint = center_y
    
    while time.time() - start_time < duration:
        # Get the largest object
        objects = tracker.get_detected_objects()
        largest = find_largest_object(objects)
        
        # Get the processed frame
        processed_frame = tracker.get_processed_frame()
        if processed_frame is None:
            time.sleep(0.1)
            continue
            
        # Create a copy for drawing
        display_frame = processed_frame.copy()
        
        # Draw the center of the frame
        cv2.circle(display_frame, (center_x, center_y), 10, (255, 255, 255), 2)
        cv2.line(display_frame, (center_x - 20, center_y), (center_x + 20, center_y), (255, 255, 255), 2)
        cv2.line(display_frame, (center_x, center_y - 20), (center_x, center_y + 20), (255, 255, 255), 2)
        
        # Calculate robot movement if an object is detected
        if largest:
            # Update PID controllers
            pid_x.update(largest['x'])
            pid_y.update(largest['y'])
            
            # Calculate movement values
            move_x = -pid_x.output  # Negative because moving right decreases error
            move_y = -pid_y.output  # Negative because moving down decreases error
            
            # Draw a line showing the direction to move
            end_x = center_x + int(move_x * 5)  # Scale for visualization
            end_y = center_y + int(move_y * 5)  # Scale for visualization
            cv2.arrowedLine(display_frame, (center_x, center_y), (end_x, end_y), (0, 255, 255), 3)
            
            # Display movement information
            cv2.putText(
                display_frame,
                f"Move X: {move_x:.2f}, Y: {move_y:.2f}",
                (10, height - 20),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.7,
                (0, 255, 255),
                2
            )
        else:
            # Display "No object detected" message
            cv2.putText(
                display_frame,
                "No object detected",
                (10, height - 20),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.7,
                (0, 0, 255),
                2
            )
        
        # Convert from BGR to RGB for matplotlib
        display_rgb = cv2.cvtColor(display_frame, cv2.COLOR_BGR2RGB)
        
        # Display the frame
        plt.figure(figsize=(10, 8))
        plt.imshow(display_rgb)
        plt.title('Object Following Simulation')
        plt.axis('off')
        
        clear_output(wait=True)
        display(plt.gcf())
        plt.close()
        
        time.sleep(0.1)

# Run the simulation for 10 seconds
simulate_following(duration=10)

### Challenge 4: Modify the Following Behavior

Try modifying the PID parameters to change how the simulated robot follows the object. What happens if you increase or decrease the proportional (p) value?

In [ ]:
# Your code here:
# Create new PID controllers with different parameters
# pid_x = PID(p=?, i=?, d=?)
# pid_y = PID(p=?, i=?, d=?)

# Run the simulation again
# simulate_following(duration=10)

## Cleanup

When you're done, make sure to stop the tracker to release the camera.

In [ ]:
# Stop the tracker
tracker.stop()
print("Tracker stopped and camera released.")

## Conclusion

In this tutorial, you've learned:

1. How to initialize and use a color tracker
2. How to change the color being tracked
3. How to get information about detected objects
4. How to create custom color ranges
5. How to simulate a simple object following behavior

These skills can be applied to various robotics projects, such as:
- Creating a robot that follows a colored ball
- Building a sorting system that separates objects by color
- Developing interactive games that respond to colored objects

Try experimenting with different colors and objects to see how the tracker responds!